In [173]:
# ПРЕПРОЦЕССИНГ https://habr.com/ru/articles/510934/
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
import pandas as pd
import csv
import os
import seaborn as sns

# https://pythonru.com/uroki/chtenie-i-zapis-fajlov-excel-xlsx-v-python
df = pd.read_excel('./Tovar_1.xlsx')  # читает только содержимое первого листа
df.head(3)  # вывод перых 3х строк на экран
#sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')   # тепловая матрица

N   article                                             Name  unit  \
0   1     6775  Бумажные полотенца в пачках SCOTT, однослойные   Кор.   
1   2     8035   Туалетная бумага в пачках HOSTESS, двухслойная  Пач.   
2   3     8550                Туалетная бумага в рулонах, SCOTT  Пач.   

   quantity   arrived   supplier  Invoice   availability  consumpt    price  \
0         27        40  "Гилана"      1654            60      46.0  1542.37   
1          8         8  "Гилана"      1655            12      11.0  1364.41   
2          9         7  "Гилана"      1674             7      11.0  1922.03   

        sum    place  total\n      sum1 Group  
0  92542.20  Склад С        7  10796.59     V  
1  16372.92  Склад С        4   5457.64     R  
2  13454.21  Склад С        9  17298.27     I

In [175]:
# Если ноль нам надо интерпретировать как отсутствие данных, то заменяем 0 на NaN
# (в противном случае этот код не выполняем).
print(' количество нулей в столбцах:')
for i in df.columns:
    print(i, df[i][df[i]==0].count())
df = df.replace(0, 'NaN')  # замена 0 на NaN во всей таблице
#df_numerical[["Объект", "Комфорт"]] = df_numerical[["Объект", "Комфорт"]].replace(0, 'NaN')
print('\n Проверяем, что замена 0 на NaN произошла')
for i in df.columns:
    print(i, df[i][df[i]==0].count())

 количество нулей в столбцах:
N  0
article 0
Name 0
unit 0
quantity  0
arrived  0
supplier 0
Invoice  0
availability 0
consumpt 0
price 0
sum 0
place 0
total
 0
sum1 0
Group 0

 Проверяем, что замена 0 на NaN произошла
N  0
article 0
Name 0
unit 0
quantity  0
arrived  0
supplier 0
Invoice  0
availability 0
consumpt 0
price 0
sum 0
place 0
total
 0
sum1 0
Group 0


In [176]:
print('Количество пустых значений до их удаления:',df.isnull().sum())
print('\nДоля пустых значений в колонке до их удаления:')
print(df.isnull().sum() / df.shape[0] * 100)

df.dropna(axis=0,inplace=True) # 0 - удаляем строки с NaN,inplace - изменить или создать новый
print('\nКоличество пустых значений после удаления NaN:\n',df.isnull().sum())
print('\nДоля пустых значений в колонке:')
print(df.isnull().sum() / df.shape[0] * 100)

Количество пустых значений до их удаления: N               0
article         0
Name            2
unit            0
quantity        0
arrived         0
supplier        8
Invoice         0
availability    0
consumpt        2
price           0
sum             0
place           0
total\n         0
sum1            0
Group           0
dtype: int64

Доля пустых значений в колонке до их удаления:
N               0.000000
article         0.000000
Name            1.438849
unit            0.000000
quantity        0.000000
arrived         0.000000
supplier        5.755396
Invoice         0.000000
availability    0.000000
consumpt        1.438849
price           0.000000
sum             0.000000
place           0.000000
total\n         0.000000
sum1            0.000000
Group           0.000000
dtype: float64

Количество пустых значений после удаления NaN:
 N               0
article         0
Name            0
unit            0
quantity        0
arrived         0
supplier        0
Invoice         0


In [ ]:
# from google.colab import drive    # подключение Google диска
# drive.mount('/content/drive')

In [177]:
df.drop_duplicates(inplace=True) # удаляем дубликаты строк, если считаем нужным
df = df[[c for c    # Перезаписываем датасет, оставляя только те колонки,
        in list(df) #  в которых больше одного уникального значения
        if len(df[c].unique()) > 1]]

In [178]:
#Разделяем датасет на два: один с качественными значениями, а другой — с количественными
df_numerical = df.select_dtypes(include = [np.number])
df_categorical = df.select_dtypes(exclude = [np.number])
#df_categorical

In [150]:
# Обработка датасета с КОЛИЧЕСТВЕННЫМИ данными
# смотрим, какие колонки могут иметь мало уникальных данных (в районе 3-7 уникальных значений).
# их целесообразно перенести в категориальные данные
print(df_numerical.nunique())   # количество уникальных значений

N               128
article          38
quantity         20
arrived          25
Invoice          36
availability     26
price            34
sum              38
total\n          19
sum1             37
dtype: int64


In [179]:
# Этот ФРАГМЕНТ ВЫПОЛНЯТЬ ТОЛЬКО ОДИН РАЗ!
# Векторизация категориальных данных(one-hot-encoding).
# При повторном запуске анализируется уже измененный в сответствии с
# one-hot-encoding датесет df, что приведет к ошибке!

def encode_and_bind(original_dataframe, feature):
    # переменная преобразуется в столько переменных 0/1, сколько существует разных значений
    dummies = pd.get_dummies(original_dataframe[feature]) # реализация one-hot-encoding
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature], axis=1)
    return(res)

# # Имена категориальных столбцов заполнить ВРУЧНУЮ
features = ["Name","unit","supplier","place","Group"]

for feature in features:
    print(feature)
    df_categorical = encode_and_bind(df_categorical, feature)
#df_categorical[0:15]

Name
unit
supplier
place
Group


In [181]:
# Совмещаем числовые и категориальные датасеты
new_df = pd.concat([df_numerical,df_categorical], axis=1)
print('Размерность (форма) объединеного датасета: ',new_df.shape)
new_df.dropna(axis=0,inplace=True) # Этот код выполняем - удаляем строки с NaN

Размерность (форма) объединеного датасета:  (128, 63)


In [182]:
# Минмаксная нормировка данных в столбцах - приведение их к диапазону [0;1]
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()   # создали объект класса MinMaxScaler
new_df = min_max_scaler.fit_transform(new_df) # еще и преобразуется в тип numpy.ndarray
print('Размерность подготовленного массива: ',new_df.shape)
print('\nФрагмент обозначения классов в строках нового массива: ')
new_df[1:7,-1-4:-1]   # -1 обозначает конечное значеие индекса

Размерность подготовленного массива:  (128, 63)

Фрагмент обозначения классов в строках нового массива: 


array([[1., 0., 0., 1.],
       [1., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]])

In [186]:
from sklearn.model_selection import train_test_split

# Заменяем возможные значения nan на числовые
new_df=np.nan_to_num(new_df, copy=True, nan=0.0, posinf=None, neginf=None)

X=new_df[:,0:-1-3]   # ВХОДНЫЕ ДАННЫЕ
Y=new_df[:,-1-4:-1]  # ВЫХОДНЫЕ ДАННЫЕ
print('Размерность X : ',X.shape,'Размерность Y: ',Y.shape)
# разделяем наборы на обучающие и тестирующие (20%)
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Размерность X :  (128, 59) Размерность Y:  (128, 4)


In [200]:
# Метод K-ближайших соседей (KNN). https://habr.com/ru/articles/801885/

import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score
from sklearn.datasets import load_iris, load_diabetes
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

model_knn = KNeighborsClassifier(20) #  используем 20 ближайших соседей
model_knn.fit(x_train, y_train)      # обучение модели
pred_res = model_knn.predict(x_test) # применение модели к тестовым данным
knn_accuracy = accuracy_score(y_test, pred_res) # расчетточности модели
#print(f'sk KNN classifier accuracy: {knn_accuracy:}')
print('Точность классификации модели на тестовых данных: ',knn_accuracy)

Точность классификации модели на тестовых данных:  0.5384615384615384


In [204]:
# Пример применения knn для набора данных  Titanic:
# https://www.kaggle.com/datasets/alexmaszanski/titanic-edited-for-my-knn-tips
# Статья с описанием https://proglib.io/p/metod-k-blizhayshih-sosedey-k-nearest-neighbour-2021-07-19
# Набор данных состоит из 15 столбцов, таких как sex (пол), fare (плата за проезд),
# p_class (класс каюты), family_size (размер семьи), и т. д. Главным признаком,
# который мы и должны предсказать в соревновании, является survived (выжил пассажир или нет)
# В примере рассматриваются только два признака: age (возраст), fare (плата за проезд).
# Загружаем данные
train_df = pd.read_csv('/content/train_data.csv')
# Избавляемся от двух столбцов без нужной информации
train_df = train_df.drop(columns=['Unnamed: 0', 'PassengerId'])
from sklearn.neighbors import KNeighborsClassifier
predictors = ['Age', 'Fare']
outcome = 'Survived'

new_record = train_df.loc[0:0, predictors]
X = train_df.loc[1:, predictors]
y = train_df.loc[1:, outcome]

kNN = KNeighborsClassifier(n_neighbors=20)
kNN.fit(X, y)
kNN.predict(new_record)
print(kNN.predict_proba(new_record))


[[0.7 0.3]]
